In [3]:
import numpy as np
import pandas as pd


import gurobipy as gp
from gurobipy import GRB

In [20]:
xlsx = pd.ExcelFile('C:/Users/shekh/Desktop/278.xlsx')




df_through = pd.read_excel(xlsx, 'through')
df_demand = pd.read_excel(xlsx, 'demand')
df_penalty = pd.read_excel(xlsx, 'penalty')
df_cost = pd.read_excel(xlsx, 'cost')
df1_supply = pd.read_excel(xlsx, 'supply')
df1_supply


,Producer,Supply
0,P1,37550
1,P2,33695
2,P3,71400
3,P4,133100
4,P5,67000


In [21]:
demand1 = df_demand.set_index('Customer').T.to_dict('records')[0]


# Completing the rest
penalty1a = df_penalty.to_numpy()
dots1, penalty1 = gp.multidict({row[0]: row[1] for row in penalty1a})
penalty1


supply1 = df1_supply.set_index('Producer').T.to_dict('records')[0]
#supply1
through1 = df_through.set_index('Depot').T.to_dict('records')[0]
#cost1
cost1a = df_cost.to_numpy()
arcs1, cost1 = gp.multidict({(row[0],row[1]): row[2] for row in cost1a})

In [22]:
model1 = gp.Model('SupplyNetworkDesign1')
flow1 = model1.addVars(arcs1, obj=cost1, name="flow1")
deficit1 = model1.addVars(dots1, obj = penalty1, name = 'deficit1')
model1.update()

# Production capacity limits

producers1 = supply1.keys()
producer_flow1 = model1.addConstrs((gp.quicksum(flow1.select(producer, '*')) <= supply1[producer]
                                 for producer in producers1), name="producer1")


# Customer demand

customers1 = demand1.keys()

customer_flow1 = model1.addConstrs((gp.quicksum(flow1.select('*', customer)+deficit1.select(customer)) == demand1[customer]
                                  for customer in customers1), name="customer1")

# Depot flow conservation

depots1 = through1.keys()
depot_flow1 = model1.addConstrs((gp.quicksum(flow1.select(depot, '*')) == gp.quicksum(flow1.select('*', depot))
                               for depot in depots1), name="depot1")

# Depot throughput

depot_capacity1 = model1.addConstrs((gp.quicksum(flow1.select('*', depot)) <= through1[depot]
                                   for depot in depots1), name="depot_capacity1")

model1.optimize()


Academic license - for non-commercial use only - expires 2021-04-13
Using license file C:\Users\shekh\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 658 rows, 28453 columns and 58199 nonzeros
Model fingerprint: 0x82f4fbd7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 1e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.05s
Presolved: 658 rows, 28453 columns, 58199 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.002e+04
 Factor NZ  : 6.466e+04 (roughly 12 MBytes of memory)
 Factor Ops : 6.776e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.42336219e+10  0.0

In [23]:
obj1 = model1.getObjective()
print('Objective Equation Function:')
obj1

print('\n Objective Function Value:')
print(obj1.getValue())

print('\n Vaccine Flow Values: \n')

product_flow1 = pd.DataFrame(columns=["From", "To", "Flow", "Cost"])
for arc in arcs1:
    if flow1[arc].x > 1e-6:
        product_flow1 = product_flow1.append({"From": arc[0], "To": arc[1], "Flow": flow1[arc].x, "Cost": flow1[arc].x*cost1[arc]}, ignore_index=True)  

product_flow1.index=[''] * len(product_flow1)
product_flow1


print('\n Defecit Values: \n')

deficit_flow1 = pd.DataFrame(columns=["Customer", "Deficit", "Penalty"])
for customer in customers1:
    if deficit1[customer].x > -1e-6:
        deficit_flow1 = deficit_flow1.append({"Customer": customer, "Deficit": deficit1[customer].x, "Penalty": deficit1[customer].x*penalty1[customer]}, ignore_index=True)

deficit_flow1.index=[''] * len(deficit_flow1)
deficit_flow1

Objective Equation Function:

 Objective Function Value:
13842269.348056749

 Vaccine Flow Values: 


 Defecit Values: 



,Customer,Deficit,Penalty
,C1,0.0,0.0
,C2,0.0,0.0
,C3,0.0,0.0
,C4,0.0,0.0
,C5,0.0,0.0
...,...,...,...
,C93,7912.2,276927.0
,C94,0.0,0.0
,C95,0.0,0.0
,C96,8865.8,310303.0


In [24]:
product_flow1

,From,To,Flow,Cost
,P1,D3,800.0,5981.961471
,P1,D9,200.0,720.669826
,P1,D19,200.0,3668.707722
,P1,D27,200.0,3473.442556
,P1,D75,200.0,2444.780656
...,...,...,...,...
,D270,C88,200.0,0.000000
,D272,C93,200.0,0.000000
,D273,C41,200.0,0.000000
,D276,C85,1000.0,0.000000


In [25]:
# writer = pd.ExcelWriter('C:/Users/PART2_278_x1.xlsx', engine='xlsxwriter')
# deficit_flow1.to_excel(writer, sheet_name='Shortage Cost')
# product_flow1.to_excel(writer, sheet_name='Transportation Cost')
# writer.save()